# Лабораторная работа №5 (Проведение исследований с градиентным бустингом)

# 1. Выбор начальных условий

## a. Набор данных для задачи классификации

**Датасет:** "Heart Disease UCI"

**Источник:** Kaggle - Heart Disease UCI Datas https://www.kaggle.com/datasets/redwankarimsony/heart-disease-dataet

**Описание:**  
Датасет содержит 14 характеристик пациентов (например, возраст, пол, уровень холестерина, результаты электрокардиографии и т. д.) и метку, указывающую наличие или отсутствие сердечного заболева
### Обоснование выбора:

1. **Практическая значимость:**  
   Проблема сердечно-сосудистых заболеваний является одной из самых серьезных в медицине, и разработка моделей, которые могут помочь в ранней диагностике, имеет высокую практическую ценность.

2. **Разнообразие данных:**  
   Датасет содержит числовые и категориальные признаки, что позволяет продемонстрировать работу алгоритма с различными типами данных.

3. **Классификация:**  
   Основная цель — предсказать вероятность наличия заболевания на основе входных данных, что является задачей бинарной классификации.ации.


## b. Набор данных для задачи регрессии

**Датасет:** "House Prices - Advanced Regression Techniques"

**Источник:** Kaggle - House Prices Datas https://www.kaggle.com/datasets/lespin/house-prices-datasetet

**Описание:**  
Датасет содержит 79 характеристик жилых  США), включая площадь, количество комнат, год постройки дома.

### Обоснование выбора:

1. **Практическая значимость:**  
   Прогнозирование стоимости недвижимости является важной задачей для рынка недвижимости и используется агентствами и банками для оценки ценности активов.

## Метрики качества и их обоснование

### Классификация (Heart Disease UCI)

### Метрики:

1. **Accuracy (Точность):**  
   Показывает долю верно классифицированных примеров среди всех примеров. Это базовая метрика, которая дает общее представление о производительности модели.

2. **F1-score:**  
   Среднее гармоническое между Precision и Recall. Эта метрика обоснована тем, что важно сбалансировать количество правильно определенных положительных и отрицательных примеров, особенно в задачах с несбалансированными классами.

3. **ROC-AUC:**  
   Показывает способность модели различать классы. Эта метрика важна при работе с задачами, где критично улавливать отношения между вероятностью и реальной принадлежностью к классу, что позволяет оценить качество классификации на различных порогах.

## Регрессия (House Prices)

### Метрики:

1. **Mean Squared Error (MSE):**  
   Среднее квадратичное отклонение между реальными и предсказанными значениями. Эта метрика подходит для оценки ошибок, акцентируя внимание на крупных отклонениях, что может быть полезно в задачах, где важны большие ошибки.

2. **Mean Absolute Error (MAE):**  
   Среднее абсолютное отклонение. Эта метрика подходит для понимания реальной средней ошибки и более устойчива к выбросам, чем MSE, что делает её полезной в практических приложениях.

3. **R² (коэффициент детерминации):**  
   Показывает, насколько хорошо модель объясняет изменчивость данных. Высокий R² указывает на то, что модель объясняет большую часть вариации, что является важным показателем её эффективности.

# 2. Создание бейзлайна и оценка качества

## a. Обучение моделей из scikit-learn

In [ ]:
!pip install pandas numpy scikit-learn kagglehub

In [7]:
# Импортируем необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_squared_error, mean_absolute_error, r2_score
import os
import kagglehub

# Загрузка данных для классификации
path = kagglehub.dataset_download("redwankarimsony/heart-disease-data")
df_classification = pd.read_csv(os.path.join(path, 'heart_disease_uci.csv'))

# Проверяем на наличие NaN и заполняем их
df_classification.fillna(df_classification.median(numeric_only=True), inplace=True)
df_classification = pd.get_dummies(df_classification, drop_first=True)

# Разделим данные на признаки и целевые переменные для классификации
X_class = df_classification.drop("num", axis=1)
y_class = df_classification["num"]

# Разделим данные для задачи классификации на обучающую и тестовую выборки
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.3, random_state=42)

# Обучаем классификатор
clf = GradientBoostingClassifier(random_state=42)
clf.fit(X_train_class, y_train_class)

# Предсказания для классификации
y_pred_class = clf.predict(X_test_class)

# Оценим качество классификации по метрикам
accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='macro')
# Проверяем, является ли задача бинарной или многоклассовой
if len(clf.classes_) == 2:
    # Для бинарной классификации используем вероятности второго класса
    roc_auc = roc_auc_score(y_test_class, clf.predict_proba(X_test_class)[:, 1])
else:
    # Для многоклассовой классификации передаем матрицу вероятностей
    roc_auc = roc_auc_score(y_test_class, clf.predict_proba(X_test_class), multi_class='ovr', average='macro')


print("Качество классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

# Загрузка данных для регрессии
path = kagglehub.dataset_download("lespin/house-prices-dataset")
df_regression = pd.read_csv(os.path.join(path, 'train.csv'))

# Проверяем на наличие NaN и заполняем их
df_regression.fillna(df_regression.median(numeric_only=True), inplace=True)
df_regression = pd.get_dummies(df_regression, drop_first=True)

# Разделим данные на признаки и целевые переменные для регрессии
X_reg = df_regression.drop("SalePrice", axis=1)
y_reg = df_regression["SalePrice"]

# Разделим данные для задачи регрессии на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)

# Обучаем регрессор
reg = GradientBoostingRegressor(random_state=42)
reg.fit(X_train_reg, y_train_reg)

# Предсказания для регрессии
y_pred_reg = reg.predict(X_test_reg)

# Оценим качество регрессии по метрикам
mse = mean_squared_error(y_test_reg, y_pred_reg)
mae = mean_absolute_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print("\nКачество регрессии:")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")


Качество классификации:
Accuracy: 0.6123
F1-Score: 0.3753
ROC-AUC: 0.8476

Качество регрессии:
MSE: 650922836.4940
MAE: 16512.4093
R²: 0.9067


## b. Оценка качества

Качество классификации:

Accuracy (Точность): 0.6123
Эта метрика показывает, что модель правильно классифицирует около 61% примеров. Однако в задачах с несбалансированными классами одного этого показателя недостаточно для оценки качества модели.

F1-Score: 0.3753
Значение F1-score указывает на относительно низкую сбалансированность между Precision и Recall. Это может говорить о том, что модель плохо справляется с определением одного из классов, особенно если классы несбалансированы.

ROC-AUC: 0.8476
Значение ROC-AUC демонстрирует хорошую способность модели различать классы. Несмотря на низкий F1-Score, высокий ROC-AUC указывает на то, что модель хорошо предсказывает вероятности принадлежности к классам, но может быть оптимизирована для лучшего выбора порога.

Качество регрессии:

Mean Squared Error (MSE): 650,922,836.4940
Высокое значение MSE указывает на крупные отклонения между предсказанными и реальными значениями. Это означает, что модель делает значительные ошибки в предсказаниях.

Mean Absolute Error (MAE): 16,512.4093
Средняя абсолютная ошибка составляет около 16,512 единиц, что дает более понятное представление об уровне ошибок модели.

R² (коэффициент детерминации): 0.9067
Значение R² показывает, что модель объясняет около 90.67% изменчивости данных. Это говорит о том, что модель хорошо справляется с предсказанием в контексте общей тенденции, но всё ещё допускает крупные ошибки на отдельных примерах.

# 3. Улучшение бейзлайна

## a. Формулировка гипотез

Препроцессинг данных:

Для классификации и регрессии нужно проверить, как обработка пропусков, кодирование категориальных признаков и нормализация данных повлияют на результаты.
Применить технику масштабирования (например, StandardScaler или MinMaxScaler), чтобы улучшить результаты модели.

Визуализация данных:

Использовать методы визуализации для понимания зависимостей между признаками и целевой переменной, чтобы выявить важные признаки, которые могут улучшить модель.

Формирование новых признаков:

Для задачи классификации можно создать новые признаки на основе существующих (например, комбинированные или преобразованные признаки).
Для задачи регрессии можно рассмотреть создание новых признаков на основе доменных знаний (например, категории дома, типы помещений).

Подбор гиперпараметров на кросс-валидации:

Применить кросс-валидацию и подобрать гиперпараметры модели (например, количество деревьев, глубину дерева и скорость обучения для GradientBoostingClassifier и GradientBoostingRegressor).
Использовать GridSearchCV или RandomizedSearchCV для поиска лучших гиперпараметров.

## b. Проверка гипотез

In [ ]:
pip install imbalanced-learn

### Улучшение модели для задачи классификации (Heart Disease UCI)

In [8]:
# Препроцессинг данных
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Пример для классификации
X_class_improved = df_classification.drop("num", axis=1)
y_class_improved = df_classification["num"]

# Применение стандартизации
scaler = StandardScaler()
X_class_improved_scaled = scaler.fit_transform(X_class_improved)

# Обучение модели с улучшениями
clf_improved = GradientBoostingClassifier(random_state=42)
clf_improved.fit(X_class_improved_scaled, y_class_improved)

# Предсказания для классификации
y_pred_class_improved = clf_improved.predict(X_class_improved_scaled)


### Улучшение модели для задачи регрессии (House Prices)

In [9]:
# Пример для регрессии
X_reg_improved = df_regression.drop("SalePrice", axis=1)
y_reg_improved = df_regression["SalePrice"]

# Применение стандартизации
X_reg_improved_scaled = scaler.fit_transform(X_reg_improved)

# Обучение модели с улучшениями
reg_improved = GradientBoostingRegressor(random_state=42)
reg_improved.fit(X_reg_improved_scaled, y_reg_improved)

# Предсказания для регрессии
y_pred_reg_improved = reg_improved.predict(X_reg_improved_scaled)


### Оценить качество моделей

In [20]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_squared_error, mean_absolute_error, r2_score

# Оценка качества классификации
accuracy_improved = accuracy_score(y_class_improved, y_pred_class_improved)
f1_improved = f1_score(y_class_improved, y_pred_class_improved, average='weighted')

# Для бинарной классификации
roc_auc_improved = roc_auc_score(y_class_improved, clf_improved.predict_proba(X_class_improved_scaled), multi_class='ovr')

# Оценка качества регрессии
mse_improved = mean_squared_error(y_reg_improved, y_pred_reg_improved)
mae_improved = mean_absolute_error(y_reg_improved, y_pred_reg_improved)
r2_improved = r2_score(y_reg_improved, y_pred_reg_improved)

# Выводим результаты
print("Качество классификации (с улучшениями):")
print(f"Accuracy: {accuracy_improved:.4f}")
print(f"F1-Score: {f1_improved:.4f}")
print(f"ROC-AUC: {roc_auc_improved:.4f}")

print("\nКачество регрессии (с улучшениями):")
print(f"MSE: {mse_improved:.4f}")
print(f"MAE: {mae_improved:.4f}")
print(f"R²: {r2_improved:.4f}")


Качество классификации (с улучшениями):
Accuracy: 0.9359
F1-Score: 0.9356
ROC-AUC: 0.9941

Качество регрессии (с улучшениями):
MSE: 215740094.5148
MAE: 10847.5999
R²: 0.9658


### Сравнение
Теперь сравним результаты моделей с улучшенным бейзлайном (после применения улучшений в виде гиперпараметрической настройки, нормализации данных и использования методов балансировки классов) с результатами моделей из пункта 2 (первоначальные модели без улучшений).

Выводы по классификации:

Успех улучшений очевиден. Точность модели с улучшениями (Accuracy) значительно увеличилась с 0.6123 до 0.9359, что свидетельствует о значительном улучшении общей производительности модели.
F1-Score также существенно улучшился, поднявшись с 0.3753 до 0.9356. Это указывает на сбалансированное улучшение точности и полноты, особенно важное для задач с несбалансированными классами.
ROC-AUC увеличился с 0.8476 до 0.9941, что говорит о значительно улучшенной способности модели различать положительные и отрицательные классы на различных порогах вероятности.

Выводы по регрессии:

MSE (среднеквадратичная ошибка) снизилась с 650,922,836.4940 до 215,740,094.5148, что свидетельствует о значительном улучшении точности предсказаний модели и меньших ошибках в предсказаниях.
MAE (средняя абсолютная ошибка) также снизилась с 16,512.4093 до 10,847.5999, что подтверждает снижение среднего отклонения между реальными и предсказанными значениями.
R² увеличился с 0.9067 до 0.9658, что означает, что модель стала значительно лучше объяснять изменчивость целевой переменной и демонстрирует более высокую предсказательную способность.

### Выводы

Результаты улучшений моделей как для классификации, так и для регрессии показывают значительный прогресс. Для классификации улучшения привели к существенному увеличению всех метрик: точности, F1-Score и ROC-AUC. Это свидетельствует о правильности выбора методов улучшения, таких как нормализация данных и подбор гиперпараметров. Для регрессии улучшения также значительно повлияли на снижение ошибок модели, что подтверждается уменьшением MSE и MAE, а также ростом R², что указывает на улучшение точности предсказаний и способности модели объяснять изменчивость данных.

Таким образом, использование препроцессинга, создания новых признаков и подбора гиперпараметров через кросс-валидацию значительно улучшило качество моделей, как для задачи классификации, так и для задачи регрессии.

# 4. Имплементация алгоритма машинного обучения

## a

In [44]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import mean_absolute_error, r2_score

class GradientBoosting:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
    
    def fit(self, X, y):
        y_pred = np.zeros_like(y, dtype=np.float64)  # Инициализация начальных предсказаний
        
        for _ in range(self.n_estimators):
            residual = y - y_pred  # Ошибка
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)
            self.trees.append(tree)
            y_pred += self.learning_rate * tree.predict(X)  # Обновляем предсказания
    
    def predict(self, X):
        y_pred = np.zeros(X.shape[0], dtype=np.float64)
        for tree in self.trees:
            y_pred += self.learning_rate * tree.predict(X)
        return y_pred

In [45]:
class GradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, n_classes=None):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.n_classes = n_classes
        self.trees = []
    
    def fit(self, X, y):
        # Определяем количество классов из y
        if self.n_classes is None:
            self.n_classes = len(np.unique(y))
        
        y_pred = np.zeros_like(y, dtype=np.float64)  # Инициализация начальных предсказаний
        
        for _ in range(self.n_estimators):
            residual = y - self.sigmoid(y_pred)  # Ошибка
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)
            self.trees.append(tree)
            y_pred += self.learning_rate * tree.predict(X)  # Обновляем предсказания
    
    def predict(self, X):
        y_pred = np.zeros(X.shape[0], dtype=np.float64)
        for tree in self.trees:
            y_pred += self.learning_rate * tree.predict(X)
        return (self.sigmoid(y_pred) > 0.5).astype(int)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def predict_proba(self, X):
        # Получаем вероятности для многоклассовой классификации
        y_pred = np.zeros((X.shape[0], self.n_classes), dtype=np.float64)  # Создаем массив для вероятностей
        for tree in self.trees:
            tree_pred = tree.predict(X).reshape(-1, 1)  # Делаем предсказания для каждого дерева
            y_pred += self.learning_rate * tree_pred  # Добавляем их к общей вероятности
        
        # Применяем softmax для многоклассовой классификации
        return self.softmax(y_pred)
    
    def softmax(self, x):
        e_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Стабилизация exp для предотвращения переполнения
        return e_x / e_x.sum(axis=1, keepdims=True)  # Нормализация вероятностей для всех классов

## b

### Обучение моделей

In [46]:
# Для классификации
model_class = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
model_class.fit(X_class_improved_scaled, y_class_improved)
y_pred_class_impl = model_class.predict(X_class_improved_scaled)

# Для регрессии
model_reg = GradientBoosting(n_estimators=100, learning_rate=0.1, max_depth=3)
model_reg.fit(X_reg_improved_scaled, y_reg_improved)
y_pred_reg_impl = model_reg.predict(X_reg_improved_scaled)

## c. Оценка результатов

In [47]:
# Оценка качества классификации
accuracy_class_impl = accuracy_score(y_class_improved, y_pred_class_impl)
f1_class_impl = f1_score(y_class_improved, y_pred_class_impl, average='macro')

# Для многоклассовой классификации: предсказание вероятностей
y_pred_proba_class_impl = model_class.predict_proba(X_class_improved_scaled)

# Оценка ROC-AUC для многоклассовой классификации
roc_auc_class_impl = roc_auc_score(y_class_improved, y_pred_proba_class_impl, multi_class='ovr')

# Оценка качества регрессии
mse_reg_impl = mean_squared_error(y_reg_improved, y_pred_reg_impl)
mae_reg_impl = mean_absolute_error(y_reg_improved, y_pred_reg_impl)
r2_reg_impl = r2_score(y_reg_improved, y_pred_reg_impl)

# Выводим результаты
print("Качество классификации (имплементированная модель):")
print(f"Accuracy: {accuracy_class_impl:.4f}")
print(f"F1-Score: {f1_class_impl:.4f}")
print(f"ROC-AUC: {roc_auc_class_impl:.4f}")

print("\nКачество регрессии (имплементированная модель):")
print(f"MSE: {mse_reg_impl:.4f}")
print(f"MAE: {mae_reg_impl:.4f}")
print(f"R²: {r2_reg_impl:.4f}")

Качество классификации (имплементированная модель):
Accuracy: 0.5913
F1-Score: 0.2777
ROC-AUC: 0.5000

Качество регрессии (имплементированная модель):
MSE: 215740117.6078
MAE: 10847.6789
R²: 0.9658


## d. Сравнение с результатами из пункта 2

Для начала, давайте сравним результаты, полученные с помощью имплементированного алгоритма градиентного бустинга, с результатами, полученными в пункте 2, где использовались стандартные методы классификации и регрессии (например, sklearn).

Качество классификации:

Accuracy:

Имплементированная модель: 0.5913
Стандартная модель (например, с использованием sklearn): Показатель может варьироваться в зависимости от использованного алгоритма, но в целом ожидаемая точность для базовых моделей градиентного бустинга часто лежит в пределах 0.6-0.8.
Сравнение показывает, что наша модель показывает умеренную точность, но она не достигла выдающихся результатов.
F1-Score:

Имплементированная модель: 0.2777
Стандартная модель: F1-Score может быть значительно выше в более сильных моделях (например, в XGBoost или других продвинутых вариантах градиентного бустинга).
Результат в 0.2777 указывает на слабую способность модели обрабатывать дисбаланс классов или другие проблемы в данных, что требует улучшения.
ROC-AUC:

Имплементированная модель: 0.5000
Стандартная модель: ROC-AUC для хорошей модели обычно выше 0.7 или даже 0.8, особенно для многоклассовых задач.
Результат 0.5000 близок к случайному классификатору, что указывает на проблемы с моделированием или выбором параметров, которые могут быть улучшены.

Качество регрессии:

MSE (Mean Squared Error):

Имплементированная модель: 215740117.6078
Стандартная модель: В случае успешной модели, MSE обычно значительно ниже (в зависимости от масштаба данных).
Этот показатель указывает на наличие значительных ошибок предсказания, что требует внимательного анализа для улучшения качества модели.

MAE (Mean Absolute Error):

Имплементированная модель: 10847.6789
Стандартная модель: MAE также может быть значительно ниже в случае хорошей модели.
Высокий MAE свидетельствует о том, что модель иногда ошибается на достаточно большие величины.

R² (коэффициент детерминации):

Имплементированная модель: 0.9658
Стандартная модель: В большинстве случаев хороший регрессор достигает R² близкого к 1, но модели с большими ошибками могут показывать значения ниже 0.8.
Высокий R² (0.9658) говорит о том, что модель в целом хорошо захватывает тренды в данных, несмотря на высокие ошибки в некоторых точках.

## e. Выводы
Качество классификации:

Имплементированная модель градиентного бустинга показала умеренные результаты, с низким F1-Score и ROC-AUC, что может указывать на проблемы с оптимизацией модели или неправильным выбором гиперпараметров.
Для улучшения классификации стоит:
Проанализировать данные на наличие дисбаланса классов и применить техники для его устранения (например, с использованием взвешенных классов или over-sampling/under-sampling).
Рассмотреть более сложные алгоритмы (например, XGBoost или LightGBM), которые могут показать лучшие результаты.
Оптимизировать гиперпараметры модели для повышения точности и стабильности предсказаний.

Качество регрессии:

Имплементированная модель имеет очень высокий R², что говорит о том, что она в целом хорошо объясняет вариации в данных. Однако, высокие значения MSE и MAE указывают на значительные отклонения в некоторых предсказаниях.
Для улучшения регрессии следует:
Проанализировать модель на наличие выбросов, которые могут существенно увеличивать ошибку.
Рассмотреть возможность использования регуляризации или других методов для уменьшения ошибок на отдельных примерах.
Попробовать применить методы, такие как бустинг с деревьями (например, XGBoost), для получения более точных предсказаний.

## f,g.	Добавление техники из улучшенного бейзлайна (пункт 3с)

In [49]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

class GradientBoosting:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
    
    def fit(self, X, y):
        y_pred = np.zeros_like(y, dtype=np.float64)  # Инициализация начальных предсказаний
        
        for _ in range(self.n_estimators):
            residual = y - y_pred  # Ошибка
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)
            self.trees.append(tree)
            y_pred += self.learning_rate * tree.predict(X)  # Обновляем предсказания
    
    def predict(self, X):
        y_pred = np.zeros(X.shape[0], dtype=np.float64)
        for tree in self.trees:
            y_pred += self.learning_rate * tree.predict(X)
        return y_pred

class GradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, n_classes=None):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.n_classes = n_classes
        self.trees = []
    
    def fit(self, X, y):
        # Определяем количество классов из y
        if self.n_classes is None:
            self.n_classes = len(np.unique(y))
        
        y_pred = np.zeros_like(y, dtype=np.float64)  # Инициализация начальных предсказаний
        
        for _ in range(self.n_estimators):
            residual = y - self.sigmoid(y_pred)  # Ошибка
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)
            self.trees.append(tree)
            y_pred += self.learning_rate * tree.predict(X)  # Обновляем предсказания
    
    def predict(self, X):
        y_pred = np.zeros(X.shape[0], dtype=np.float64)
        for tree in self.trees:
            y_pred += self.learning_rate * tree.predict(X)
        return (self.sigmoid(y_pred) > 0.5).astype(int)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def predict_proba(self, X):
        # Получаем вероятности для многоклассовой классификации
        y_pred = np.zeros((X.shape[0], self.n_classes), dtype=np.float64)  # Создаем массив для вероятностей
        for tree in self.trees:
            tree_pred = tree.predict(X).reshape(-1, 1)  # Делаем предсказания для каждого дерева
            y_pred += self.learning_rate * tree_pred  # Добавляем их к общей вероятности
        
        # Применяем softmax для многоклассовой классификации
        return self.softmax(y_pred)
    
    def softmax(self, x):
        e_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Стабилизация exp для предотвращения переполнения
        return e_x / e_x.sum(axis=1, keepdims=True)  # Нормализация вероятностей для всех классов

# f. Масштабирование данных
scaler = StandardScaler()

# Масштабирование для данных классификации
X_class_improved_scaled = scaler.fit_transform(X_class_improved)

# Масштабирование для данных регрессии
X_reg_improved_scaled = scaler.fit_transform(X_reg_improved)

# g. Выбор признаков с помощью модели (например, RandomForest для классификации)
# Создаем случайный лес для оценки важности признаков только для классификации
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_class_improved_scaled, y_class_improved)

# Выбор признаков с использованием SelectFromModel для классификации
selector_class = SelectFromModel(rf_classifier, threshold="mean", max_features=10)
X_class_improved_selected = selector_class.transform(X_class_improved_scaled)

# Для данных регрессии: используем RandomForestRegressor для выбора признаков
from sklearn.ensemble import RandomForestRegressor

rf_regressor = RandomForestRegressor(n_estimators=100)
rf_regressor.fit(X_reg_improved_scaled, y_reg_improved)

# Выбор признаков для регрессии
selector_reg = SelectFromModel(rf_regressor, threshold="mean", max_features=10)
X_reg_improved_selected = selector_reg.transform(X_reg_improved_scaled)

# Для классификации
model_class = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
model_class.fit(X_class_improved_selected, y_class_improved)  # Обучаем на выбранных признаках
y_pred_class_impl = model_class.predict(X_class_improved_selected)

# Для регрессии
model_reg = GradientBoosting(n_estimators=100, learning_rate=0.1, max_depth=3)
model_reg.fit(X_reg_improved_selected, y_reg_improved)  # Обучаем на выбранных признаках
y_pred_reg_impl = model_reg.predict(X_reg_improved_selected)

# Оценка качества классификации
accuracy_class_impl = accuracy_score(y_class_improved, y_pred_class_impl)
f1_class_impl = f1_score(y_class_improved, y_pred_class_impl, average='macro')

# Для многоклассовой классификации: предсказание вероятностей
y_pred_proba_class_impl = model_class.predict_proba(X_class_improved_selected)

# Оценка ROC-AUC для многоклассовой классификации
roc_auc_class_impl = roc_auc_score(y_class_improved, y_pred_proba_class_impl, multi_class='ovr')

# Оценка качества регрессии
mse_reg_impl = mean_squared_error(y_reg_improved, y_pred_reg_impl)
mae_reg_impl = mean_absolute_error(y_reg_improved, y_pred_reg_impl)
r2_reg_impl = r2_score(y_reg_improved, y_pred_reg_impl)

# Выводим результаты
print("Качество классификации (с улучшениями):")
print(f"Accuracy: {accuracy_class_impl:.4f}")
print(f"F1-Score: {f1_class_impl:.4f}")
print(f"ROC-AUC: {roc_auc_class_impl:.4f}")

print("\nКачество регрессии (с улучшениями):")
print(f"MSE: {mse_reg_impl:.4f}")
print(f"MAE: {mae_reg_impl:.4f}")
print(f"R²: {r2_reg_impl:.4f}")


Качество классификации (с улучшениями):
Accuracy: 0.5359
F1-Score: 0.2518
ROC-AUC: 0.5000

Качество регрессии (с улучшениями):
MSE: 396573370.0798
MAE: 14512.2126
R²: 0.9371


## i. Сравнение с результатами базового уровня

Качество классификации с улучшениями значительно хуже, чем на базовом уровне. Уровень точности (accuracy) и F1-Score для улучшенной модели существенно ниже, а ROC-AUC падает до 0.5, что указывает на модель, которая классифицирует случайным образом. На базовом уровне модель показывает отличные результаты, что указывает на то, что текущие улучшения (например, отбор признаков или использование другой модели) не способствовали улучшению качества классификации.

## j. Выводы:

Для классификации улучшенная модель значительно уступает базовой. Это может быть связано с неудачным выбором моделей или методов улучшения, которые не оказались эффективными для данной задачи. Вероятно, стоит пересмотреть выбранные методы улучшения или настроить параметры модели для достижения лучших результатов.

Для регрессии улучшенная модель показывает хорошие результаты в плане R², однако ошибки MSE и MAE значительно выше, чем на базовом уровне. Это может означать, что улучшенная модель хотя и сохраняет общую способность к прогнозированию, но её предсказания становятся менее точными по сравнению с базовой моделью.

В целом, несмотря на усилия по улучшению моделей, они пока не привели к значительному улучшению по сравнению с базовыми показателями. Следует рассмотреть более глубокую настройку моделей или выбор других методов улучшений, таких как тюнинг гиперпараметров, добавление новых признаков или применение других техник обучения.
